In [1]:
import cv2
import time
#from dotenv import load_dotenv
import os

### Test RTSP

In [5]:
def preprocess_frame(frame, target_size=640):
    """
    Принимает кадр размером (720, 1280, 3), уменьшает до (360, 640, 3),
    затем добавляет черные полосы сверху и снизу, чтобы получить (640, 640, 3).

    Возвращает:
        padded_frame: кадр для инференса (640, 640, 3)
    """

    resized_frame = cv2.resize(frame, (640, 360))  # (width, height)

    pad_size = (target_size - resized_frame.shape[0]) // 2  # (640 - 360) // 2 = 140
    top_pad = pad_size
    bottom_pad = target_size - resized_frame.shape[0] - top_pad

    padded_frame = cv2.copyMakeBorder(
        resized_frame,
        top=top_pad,
        bottom=bottom_pad,
        left=0,
        right=0,
        borderType=cv2.BORDER_CONSTANT,
        value=(0, 0, 0)  # черный цвет
    )


    return padded_frame

In [ ]:
def capture_rtsp_stream(rtsp_url):
    cap = cv2.VideoCapture(rtsp_url)

    if not cap.isOpened():
        print("Ошибка: Не удалось открыть видеопоток.")
        return

    frame_counter = 0
    start_time = time.time()
    fps_display_interval = 1.0
    last_fps_update = time.time()

    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Ошибка: Не удалось получить кадр из видеопотока.")
            break
        frame_counter += 1
        current_time = time.time()
        elapsed_time = current_time - start_time
        if elapsed_time >= fps_display_interval:
            fps = frame_counter / elapsed_time
            frame_counter = 0
            start_time = current_time

            last_fps_update = current_time
        frame = preprocess_frame(frame)
        text = f"FPS: {fps:.2f}" if 'fps' in locals() else "FPS: N/A"
        (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        cv2.rectangle(frame, (10, 10), (10 + text_width, 10 + text_height + 10), (0, 0, 0), -1)  # Закрашиваем фон
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("RTSP Stream", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [11]:
# load_dotenv()
# rtsp_url = os.getenv("RTSP_URL")
rtsp_url = "rtsp://192.168.1.100/live/ch00_0"

if not rtsp_url:
    print("Ошибка: Переменная окружения RTSP_URL не установлена в файле .env.")
else:
    capture_rtsp_stream(rtsp_url)

(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)
(640, 640, 3)


### TEST ONVIF

In [1]:
from onvif import ONVIFCamera

# Настройки камеры
CAMERA_IP = '192.168.1.102'  # Замените на IP вашей камеры
CAMERA_PORT = 8899            # Порт ONVIF (обычно 80 или 8080)
CAMERA_USER = 'admin'        # Логин
CAMERA_PASSWORD = '' # Пароль

#rtsp://192.168.1.102/ptz?direction=up

In [2]:
# Создаем объект камеры
camera = ONVIFCamera(CAMERA_IP, CAMERA_PORT, CAMERA_USER, CAMERA_PASSWORD)

In [4]:
# Инициализация сервиса PTZ
ptz_service = camera.create_ptz_service()
media_service = camera.create_media_service()

# Получаем профиль камеры (необходим для PTZ-команд)
profiles = media_service.GetProfiles()
profile_token = profiles[0].token

# Функция для перемещения камеры
def move_camera(pan, tilt, zoom=0):
    """
    Перемещает камеру в заданном направлении.
    :param pan: Горизонтальное перемещение (-1.0 до 1.0)
    :param tilt: Вертикальное перемещение (-1.0 до 1.0)
    :param zoom: Масштабирование (0.0 до 1.0)
    """
    request = ptz_service.create_type('ContinuousMove')
    request.ProfileToken = profile_token
    request.Velocity = {
        'PanTilt': {'x': pan, 'y': tilt},
        'Zoom': {'x': zoom}
    }
    ptz_service.ContinuousMove(request)

# Примеры команд для перемещения камеры
if __name__ == "__main__":
    try:
        print("Перемещение камеры вверх...")
        move_camera(pan=0, tilt=0.5)  # Движение вверх

        print("Ожидание 2 секунд...")
        import time
        time.sleep(2)

        print("Перемещение камеры вправо...")
        move_camera(pan=0.5, tilt=0)  # Движение вправо

        time.sleep(2)

        print("Остановка движения...")
        ptz_service.Stop({'ProfileToken': profile_token})  # Остановка движения

    except Exception as e:
        print(f"Произошла ошибка: {e}")

Перемещение камеры вверх...
Ожидание 2 секунд...
Перемещение камеры вправо...
Остановка движения...


### Test YOLO 11 real time

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model = YOLO("yolov11_dogs_pose/yolo11n-pose_harry.pt")

In [3]:
def preprocess_frame(frame, target_size=640):
    """
    Принимает кадр размером (720, 1280, 3), уменьшает до (360, 640, 3),
    затем добавляет черные полосы сверху и снизу, чтобы получить (640, 640, 3).

    Возвращает:
        padded_frame: кадр для инференса (640, 640, 3)
    """

    resized_frame = cv2.resize(frame, (640, 360))  # (width, height)

    pad_size = (target_size - resized_frame.shape[0]) // 2  # (640 - 360) // 2 = 140
    top_pad = pad_size
    bottom_pad = target_size - resized_frame.shape[0] - top_pad

    padded_frame = cv2.copyMakeBorder(
        resized_frame,
        top=top_pad,
        bottom=bottom_pad,
        left=0,
        right=0,
        borderType=cv2.BORDER_CONSTANT,
        value=(0, 0, 0)  # черный цвет
    )


    return padded_frame

In [4]:
import cv2
import numpy as np

def draw_pose(frame, keypoints, draw_skeleton=False, conf_threshold=0.15):
    """
    Отрисовывает pose-информацию на кадре.

    Параметры:
        frame: np.ndarray (720, 1280, 3)
        keypoints: torch.Tensor (N, 19, 3) из res.keypoints.xy
        scale: float — масштаб по ширине (640 / 1280)
        vertical_scale: float — масштаб по высоте (360 / 720)
        top_pad: int — верхняя черная полоса после preprocess_frame
        draw_skeleton: bool — рисовать ли скелет
        conf_threshold: float — порог уверенности для отрисовки
    """
    h, w = frame.shape[:2]  # должно быть 720, 1280

    # Скелет — список пар точек для соединения
    skeleton = [
        (0, 1), (1, 2), (2, 3), (3, 4),  # голова и шея
        (5, 6), (6, 7), (7, 8), (8, 9),  # передние лапы
        (10, 11), (11, 12), (12, 13), (13, 14),  # задние лапы
        (4, 15), (15, 16), (16, 17), (17, 18)  # хвост
    ]

    # Конвертируем keypoints в numpy
    keypoints_np = keypoints.xy.cpu().numpy() 
    conf_np = keypoints.conf.cpu().numpy() 
    for p, person_kps in enumerate(keypoints_np):
        # Отрисовка точек
        for i, (x, y) in enumerate(person_kps):
            if 0 <= x < w and 0 <= y < h and conf_np[p][i] > conf_threshold:
                # Рисуем точку
                cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
                # Рисуем номер точки
                cv2.putText(frame, str(i), (int(x)+5, int(y)), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (255, 255, 255), 1, lineType=cv2.LINE_AA)

        # Отрисовка скелета
        if draw_skeleton:
            for start, end in skeleton:
                x1, y1 = person_kps[start]
                x2, y2 = person_kps[end]
                if (0 <= x1 < w and 0 <= y1 < h) and (0 <= x2 < w and 0 <= y2 < h):
                    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

    return frame

In [ ]:
box_conf = 0.3
confs = []

rtsp_url = "rtsp://192.168.1.100/live/ch00_0"
cap = cv2.VideoCapture(rtsp_url)

# Проверка успешного открытия потока
if not cap.isOpened():
    print("Ошибка: Не удалось открыть RTSP-поток")
    exit()
i = 5
n = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ошибка: Не удалось получить кадр.")
        break
    if i % 5:
        frame = preprocess_frame(frame)
        results = model(frame)
        if results:
            res = results[0]
            if res.boxes:
                keypoints = res.keypoints
                conf = res.boxes.conf.cpu().numpy()[0]
                if keypoints is not None and conf > box_conf:
                    frame = draw_pose(frame, keypoints)

        cv2.imshow('YOLOv11 Pose RTSP', frame)

        # Выход по нажатию 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    i += 1

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()


0: 640x640 2 dogs, 5.7ms
Speed: 4.7ms preprocess, 5.7ms inference, 123.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 dog, 5.9ms
Speed: 1.0ms preprocess, 5.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 dogs, 5.9ms
Speed: 1.0ms preprocess, 5.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 dogs, 5.9ms
Speed: 1.0ms preprocess, 5.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 5.9ms
Speed: 0.9ms preprocess, 5.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 6.1ms
Speed: 1.0ms preprocess, 6.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 5.8ms
Speed: 1.1ms preprocess, 5.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 5.8ms
Speed: 1.0ms preprocess, 5.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 5.7

### save

In [51]:
!pwd

/home/vladislav/vlad_projects/DogVision/notebooks


In [92]:
import cv2
import numpy as np
import os
from pathlib import Path
from ultralytics import YOLO

output_frames_dir = "./yolov11_dogs_pose/datasets/new_harry_cvat/images/train/"
output_labels_dir = "./yolov11_dogs_pose/datasets/new_harry_cvat/labels/train/"

rtsp_url = "rtsp://192.168.1.100/live/ch00_0"
model = YOLO("yolov11_dogs_pose/yolo11n-pose_harry.pt")

In [93]:
def preprocess_frame(frame, target_size=640):
    """
    Принимает кадр размером (720, 1280, 3), уменьшает до (360, 640, 3),
    затем добавляет черные полосы сверху и снизу, чтобы получить (640, 640, 3).

    Возвращает:
        padded_frame: кадр для инференса (640, 640, 3)
    """

    resized_frame = cv2.resize(frame, (640, 360))  # (width, height)

    pad_size = (target_size - resized_frame.shape[0]) // 2  # (640 - 360) // 2 = 140
    top_pad = pad_size
    bottom_pad = target_size - resized_frame.shape[0] - top_pad

    padded_frame = cv2.copyMakeBorder(
        resized_frame,
        top=top_pad,
        bottom=bottom_pad,
        left=0,
        right=0,
        borderType=cv2.BORDER_CONSTANT,
        value=(0, 0, 0)  # черный цвет
    )


    return padded_frame

In [94]:
cap = cv2.VideoCapture(rtsp_url)

# Проверка
if not cap.isOpened():
    print("Ошибка: Не удалось открыть RTSP-поток")
    exit()

frame_count = 0
save_interval = 25
box_conf = 0.3 
n = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ошибка: Не удалось получить кадр.")
        break

    frame_count += 1

    # Сохраняем каждый 30-й кадр
    if frame_count % save_interval == 0:
        frame = preprocess_frame(frame)
        h, w = frame.shape[:2]

        results = model(frame, verbose=False)
        
        # Собираем все объекты с conf >= box_conf
        valid_objects = []
        for res in results:
            if res.keypoints is not None:
                for box, kps in zip(res.boxes, res.keypoints):
                    conf = box.conf.item()
                    if conf >= box_conf:
                        valid_objects.append((conf, box, kps))
        
        # Выбираем объект с максимальным conf
        if valid_objects:
            max_conf_obj = max(valid_objects, key=lambda x: x[0])
            conf, box, kps = max_conf_obj

            # Получаем координаты bounding box
            x1, y1, x2, y2 = box.xyxy.cpu().numpy()[0]
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            box_width = (x2 - x1) / w
            box_height = (y2 - y1) / h

            # Получаем координаты 19 ключевых точек
            kps_xy = kps.xy.cpu().numpy()[0]

            # Проверяем, что точек ровно 19
            if len(kps_xy) == 19:
                # Формируем строку в формате YOLO Pose
                label_str = f"0 {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f} "
                
                # Добавляем точки с видимостью=1
                for x, y in kps_xy:
                    nx = x / w
                    ny = y / h
                    label_str += f"{nx:.6f} {ny:.6f} 2 "
                
                # Сохраняем кадр и разметку
                img_filename = f"frame_{n:06d}.jpg"
                label_filename = f"frame_{n:06d}.txt"

                img_path = os.path.join(output_frames_dir, img_filename)
                label_path = os.path.join(output_labels_dir, label_filename)

                cv2.imwrite(img_path, frame)
                with open(label_path, 'w', encoding="utf-8") as f:
                    f.write(label_str.strip())
                
                n += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
with open("lo.txt", 'w', encoding="utf-8") as f:
    f.write(label_str.strip())

In [67]:
import os

label_dir = os.path.dirname(label_path)
if not os.path.exists(label_dir):
    os.makedirs(label_dir, exist_ok=True)

PermissionError: [Errno 13] Отказано в доступе: '/home/vlad_projects'

In [72]:
os.listdir(output_frames_dir)

FileNotFoundError: [Errno 2] Нет такого файла или каталога: '/home/vlad_projects/DogVision/notebooks/yolov11_dogs_pose/datasets/new_harry_cvat/images'